# init

## import Python modules

In [1]:
import pandas, IPython.core.display

## open file

In [2]:
src = pandas.ExcelFile('data/real/2017/CAM data from iPads/2017 CAM data from iPads.xlsx')

In [3]:
src.sheet_names

['2017 CAM data Erl',
 'schema (WIP reverse engineer)',
 '2017 CAM iPad data Tyler',
 'Combined iPad 2017 CAM data']

## select sheets

In [4]:
sheets_to_process = {
    sheet_name.split(' ')[-1]: src.parse(sheet_name)
    for sheet_name in src.sheet_names
    if any(
        (
            person_name in sheet_name
            for person_name
            in ['Erl', 'Tyler']
        )
    )
}
sheets_to_process.keys()

dict_keys(['Erl', 'Tyler'])

# cleanup

## normalize column names

In [5]:
sheets_needing_column_rename = (
    (sheet_name, sheet)
    for sheet_name, sheet
    in sheets_to_process.items()
    if any(
        (str(column).find(' ')
         for column in sheet.columns)
    )
)
sheets_to_process = {
    sheet_name: sheet.rename(
        mapper=lambda x: str(x).split(' ')[0],
        axis='columns')
    for sheet_name, sheet in sheets_needing_column_rename
}

## concatenate sheets

In [6]:
df = pandas.concat(sheets_to_process,
                   names=['worksheet_name', 'index'])

In [7]:
# correct worksheet_row index for offset
df = df.reset_index()
df['worksheet_row'] = df.pop('index') + 2

## reindex columns

In [8]:
names = ['field', 'scope', 'variable']
columns = (
    ('fields__oSets__date',                              'set', 'date'),
    ('fields__oSets__oPoints__id',                       'point', 'id'),
#     ('fields__oSets__oPoints__name',                     'point', 'name'), 
    ('fields__oSets__oPoints__observations__id',         'observation', 'id'),
#     ('fields__oSets__oPoints__observations__name',       'observation', 'name'),

    ('fields__oSets__oPoints__observations__enum',       'point', 'natural_enemy_subtotal'),
    ('fields__oSets__oPoints__observations__|',          'natural_enemy_count', 'name'),
    ('fields__oSets__oPoints__observations__|__number',  'natural_enemy_count', 'count'),
    ('fields__oSets__oPoints__observations__complete',   'observation', 'complete'),
    ('fields__oSets__oPoints__observations__disabled',   'observation', 'disabled'),
    ('fields__oSets__oPoints__observations__a1__number', 'aphid_count', 'a1'),
    ('fields__oSets__oPoints__observations__a2__number', 'aphid_count', 'a2'),
    ('fields__oSets__oPoints__observations__a3__number', 'aphid_count', 'a3'),
    ('fields__oSets__oPoints__observations__anum',       'observation', 'aphid_subtotal'),
    ('fields__oSets__obsName',                           'set', 'obsName'),
    ('fields__oSets__desc',                              'set', 'desc'),
    ('fields__oSets__growthStage',                       'set', 'growthStage'),
    ('fields__oSets__results',                           'set_results', 'results'),
    ('fields__oSets__totalA1',                           'set_results', 'totalA1'),
    ('fields__oSets__totalA2',                           'set_results', 'totalA2'),
    ('fields__oSets__totalA3',                           'set_results', 'totalA3'),
    ('fields__oSets__totalA4',                           'set_results', 'totalA4'),
    ('fields__oSets__totalSets',                         'set_results', 'totalSets'),
    ('fields__oSets__completeSets',                      'set_results', 'completeSets'),
    ('fields__client__company',                          'field', 'client_company'),
    ('fields__client__displayText',                      'field', 'client_displayText'),
    ('fields__client__fname',                            'field', 'client_fname'),
    ('fields__client__lname',                            'field', 'client_lname'),
    ('fields__client__name',                             'field', 'client_name'),
    ('fields__crop',                                     'field', 'crop'),
    ('fields__date',                                     'field', 'date'),
    ('fields__desc',                                     'field', 'desc'),
    ('fields__image',                                    'field', 'image'),
    ('fields__name',                                     'field', 'name'),
)
columns = pandas.MultiIndex.from_tuples(columns, names=names)
df = df.reindex(columns=columns, level='field')
df.columns = df.columns.droplevel(level='field')

## convert_datetime

In [9]:
for column in [('set','date'), ('field','date')]:
    df[column] = pandas.to_datetime(df[column], infer_datetime_format=True)

## forward fill

In [10]:
fill_columns = [
    ('point', 'id'),
    ('observation', 'id'),
] + list(df[['field', 'set']].columns.values)
df.loc[:, fill_columns] = df.loc[:, fill_columns].ffill()

In [11]:
df = df.set_index(([('set', 'date'), ('point', 'id')])).sort_index().reset_index()
df

scope                        set point observation                  point  \
variable                    date    id          id natural_enemy_subtotal   
0        2017-07-14 12:31:24.194   0.0         0.0                    NaN   
1        2017-07-14 12:31:24.194   0.0         1.0                    NaN   
2        2017-07-14 12:31:24.194   0.0         2.0                    NaN   
3        2017-07-14 12:31:24.194   0.0         3.0                    NaN   
4        2017-07-14 12:31:24.194   0.0         4.0                    NaN   
5        2017-07-14 12:31:24.194   0.0         5.0                    2.0   
6        2017-07-14 12:31:24.194   0.0         5.0                    NaN   
7        2017-07-14 12:31:24.194   0.0         5.0                    NaN   
8        2017-07-14 12:31:24.194   0.0         5.0                    NaN   
9        2017-07-14 12:31:24.194   0.0         5.0                    NaN   
10       2017-07-14 12:31:24.194   0.0         5.0                    NaN   
11       2017-07-14 12:31:24.194   0.0         5.0                    NaN   
12       2017-07-14 12:31:24.194   0.0         5.0                    NaN   
13       2017-07-14 12:31:24.194   0.0         5.0                    NaN   
14       2017-07-14 12:31:24.194   1.0         0.0                    NaN   
15       2017-07-14 12:31:24.194   1.0         1.0                    NaN   
16       2017-07-14 12:31:24.194   1.0         2.0                    NaN   
17       2017-07-14 12:31:24.194   1.0         3.0                    NaN   
18       2017-07-14 12:31:24.194   1.0         4.0                    NaN   
19       2017-07-14 12:31:24.194   1.0         5.0                    0.0   
20       2017-07-14 12:31:24.194   1.0         5.0                    NaN   
21       2017-07-14 12:31:24.194   1.0         5.0                    NaN   
22       2017-07-14 12:31:24.194   1.0         5.0                    NaN   
23       2017-07-14 12:31:24.194   1.0         5.0                    NaN   
24       2017-07-14 12:31:24.194   1.0         5.0                    NaN   
25       2017-07-14 12:31:24.194   1.0         5.0                    NaN   
26       2017-07-14 12:31:24.194   1.0         5.0                    NaN   
27       2017-07-14 12:31:24.194   1.0         5.0                    NaN   
28       2017-07-14 12:31:24.194   2.0         0.0                    NaN   
29       2017-07-14 12:31:24.194   2.0         1.0                    NaN   
...                          ...   ...         ...                    ...   
4660     2017-08-25 13:05:14.062   7.0         5.0                    NaN   
4661     2017-08-25 13:05:14.062   7.0         5.0                    NaN   
4662     2017-08-25 13:05:14.062   8.0         0.0                    NaN   
4663     2017-08-25 13:05:14.062   8.0         1.0                    NaN   
4664     2017-08-25 13:05:14.062   8.0         2.0                    NaN   
4665     2017-08-25 13:05:14.062   8.0         3.0                    NaN   
4666     2017-08-25 13:05:14.062   8.0         4.0                    NaN   
4667     2017-08-25 13:05:14.062   8.0         5.0                    0.0   
4668     2017-08-25 13:05:14.062   8.0         5.0                    NaN   
4669     2017-08-25 13:05:14.062   8.0         5.0                    NaN   
4670     2017-08-25 13:05:14.062   8.0         5.0                    NaN   
4671     2017-08-25 13:05:14.062   8.0         5.0                    NaN   
4672     2017-08-25 13:05:14.062   8.0         5.0                    NaN   
4673     2017-08-25 13:05:14.062   8.0         5.0                    NaN   
4674     2017-08-25 13:05:14.062   8.0         5.0                    NaN   
4675     2017-08-25 13:05:14.062   8.0         5.0                    NaN   
4676     2017-08-25 13:05:14.062   9.0         0.0                    NaN   
4677     2017-08-25 13:05:14.062   9.0         1.0                    NaN   
4678     2017-08-25 13:05:14.062   9.0         2.0                    NaN

## split frame into natural enemies, observations

In [12]:
base_df = df.loc[df[('observation', 'aphid_subtotal')].notna() | df[('point', 'natural_enemy_subtotal')].notna(), ['field', 'set', 'point', 'observation', 'set_results']]
base_df

scope                        set point observation                  point  \
variable                    date    id          id natural_enemy_subtotal   
0        2017-07-14 12:31:24.194   0.0         0.0                    NaN   
1        2017-07-14 12:31:24.194   0.0         1.0                    NaN   
2        2017-07-14 12:31:24.194   0.0         2.0                    NaN   
3        2017-07-14 12:31:24.194   0.0         3.0                    NaN   
4        2017-07-14 12:31:24.194   0.0         4.0                    NaN   
5        2017-07-14 12:31:24.194   0.0         5.0                    2.0   
14       2017-07-14 12:31:24.194   1.0         0.0                    NaN   
15       2017-07-14 12:31:24.194   1.0         1.0                    NaN   
16       2017-07-14 12:31:24.194   1.0         2.0                    NaN   
17       2017-07-14 12:31:24.194   1.0         3.0                    NaN   
18       2017-07-14 12:31:24.194   1.0         4.0                    NaN   
19       2017-07-14 12:31:24.194   1.0         5.0                    0.0   
28       2017-07-14 12:31:24.194   2.0         0.0                    NaN   
29       2017-07-14 12:31:24.194   2.0         1.0                    NaN   
30       2017-07-14 12:31:24.194   2.0         2.0                    NaN   
31       2017-07-14 12:31:24.194   2.0         3.0                    NaN   
32       2017-07-14 12:31:24.194   2.0         4.0                    NaN   
33       2017-07-14 12:31:24.194   2.0         5.0                    0.0   
42       2017-07-14 12:31:24.194   3.0         0.0                    NaN   
43       2017-07-14 12:31:24.194   3.0         1.0                    NaN   
44       2017-07-14 12:31:24.194   3.0         2.0                    NaN   
45       2017-07-14 12:31:24.194   3.0         3.0                    NaN   
46       2017-07-14 12:31:24.194   3.0         4.0                    NaN   
47       2017-07-14 12:31:24.194   3.0         5.0                    0.0   
56       2017-07-14 12:31:24.194   4.0         0.0                    NaN   
57       2017-07-14 12:31:24.194   4.0         1.0                    NaN   
58       2017-07-14 12:31:24.194   4.0         2.0                    NaN   
59       2017-07-14 12:31:24.194   4.0         3.0                    NaN   
60       2017-07-14 12:31:24.194   4.0         4.0                    NaN   
61       2017-07-14 12:31:24.194   4.0         5.0                    0.0   
...                          ...   ...         ...                    ...   
4620     2017-08-25 13:05:14.062   5.0         0.0                    NaN   
4621     2017-08-25 13:05:14.062   5.0         1.0                    NaN   
4622     2017-08-25 13:05:14.062   5.0         2.0                    NaN   
4623     2017-08-25 13:05:14.062   5.0         3.0                    NaN   
4624     2017-08-25 13:05:14.062   5.0         4.0                    NaN   
4625     2017-08-25 13:05:14.062   5.0         5.0                    0.0   
4634     2017-08-25 13:05:14.062   6.0         0.0                    NaN   
4635     2017-08-25 13:05:14.062   6.0         1.0                    NaN   
4636     2017-08-25 13:05:14.062   6.0         2.0                    NaN   
4637     2017-08-25 13:05:14.062   6.0         3.0                    NaN   
4638     2017-08-25 13:05:14.062   6.0         4.0                    NaN   
4639     2017-08-25 13:05:14.062   6.0         5.0                    0.0   
4648     2017-08-25 13:05:14.062   7.0         0.0                    NaN   
4649     2017-08-25 13:05:14.062   7.0         1.0                    NaN   
4650     2017-08-25 13:05:14.062   7.0         2.0                    NaN   
4651     2017-08-25 13:05:14.062   7.0         3.0                    NaN   
4652     2017-08-25 13:05:14.062   7.0         4.0                    NaN   
4653     2017-08-25 13:05:14.062   7.0         5.0                    0.0   
4662     2017-08-25 13:05:14.062   8.0         0.0                    NaN

In [13]:
aphid_df = df.loc[: ,['aphid_count']].fillna(0)
aphid_df

scope    aphid_count          
variable          a1   a2   a3
0                7.0  0.0  0.0
1                0.0  0.0  0.0
2                0.0  0.0  0.0
3                0.0  0.0  0.0
4                0.0  0.0  0.0
5                0.0  0.0  0.0
6                0.0  0.0  0.0
7                0.0  0.0  0.0
8                0.0  0.0  0.0
9                0.0  0.0  0.0
10               0.0  0.0  0.0
11               0.0  0.0  0.0
12               0.0  0.0  0.0
13               0.0  0.0  0.0
14               0.0  0.0  0.0
15               0.0  0.0  0.0
16               0.0  0.0  0.0
17               0.0  0.0  0.0
18               0.0  0.0  0.0
19               0.0  0.0  0.0
20               0.0  0.0  0.0
21               0.0  0.0  0.0
22               0.0  0.0  0.0
23               0.0  0.0  0.0
24               0.0  0.0  0.0
25               0.0  0.0  0.0
26               0.0  0.0  0.0
27               0.0  0.0  0.0
28               0.0  0.0  0.0
29               0.0  0.0  0.0
...              ...  ...  ...
4660             0.0  0.0  0.0
4661             0.0  0.0  0.0
4662             0.0  0.0  0.0
4663             0.0  0.0  0.0
4664             0.0  0.0  0.0
4665             0.0  0.0  0.0
4666             0.0  0.0  0.0
4667             0.0  0.0  0.0
4668             0.0  0.0  0.0
4669             0.0  0.0  0.0
4670             0.0  0.0  0.0
4671             0.0  0.0  0.0
4672             0.0  0.0  0.0
4673             0.0  0.0  0.0
4674             0.0  0.0  0.0
4675             0.0  0.0  0.0
4676             0.0  0.0  0.0
4677             0.0  0.0  0.0
4678             2.0  3.0  0.0
4679             0.0  0.0  0.0
4680             0.0  0.0  0.0
4681             0.0  0.0  0.0
4682             0.0  0.0  0.0
4683             0.0  0.0  0.0
4684             0.0  0.0  0.0
4685             0.0  0.0  0.0
4686             0.0  0.0  0.0
4687             0.0  0.0  0.0
4688             0.0  0.0  0.0
4689             0.0  0.0  0.0

[4690 rows x 3 columns]

In [14]:
ne_columns = list(df[[('set', 'date'), ('point', 'id'), ('observation', 'complete')]].columns) + list(df[['natural_enemy_count']].columns)
ne = df.loc[:, ne_columns]
ne = ne.dropna(subset=df[[('natural_enemy_count', 'name')]].columns)
ne = ne.reset_index()
ne = ne.set_index([('set', 'date'), ('point', 'id'), ('natural_enemy_count', 'name')]).fillna(0)

## natural enemy transpose

In [15]:
ne2 = ne.unstack().reset_index()
ne2['index_base'] = ne2.pop(('index','','e1'))

In [16]:
oc = ne2.pop(('observation', 'complete','e1'))
ne2[('observation', 'complete', '')] = oc

In [17]:
ne3 = ne2.set_index('index_base').loc[df[('observation', 'complete')] > 0, ['set','point','natural_enemy_count']]
ne3.index.name = None
ne3

scope                                               set point  \
variable                                           date    id   
('natural_enemy_count', 'name')                                 
5                               2017-07-14 12:31:24.194   0.0   
19                              2017-07-14 12:31:24.194   1.0   
33                              2017-07-14 12:31:24.194   2.0   
47                              2017-07-14 12:31:24.194   3.0   
61                              2017-07-14 12:31:24.194   4.0   
75                              2017-07-18 10:31:22.263   0.0   
89                              2017-07-18 10:31:22.263   1.0   
103                             2017-07-18 10:31:22.263   2.0   
117                             2017-07-18 10:31:22.263   3.0   
131                             2017-07-18 10:31:22.263   4.0   
145                             2017-07-18 10:51:30.195   0.0   
159                             2017-07-18 10:51:30.195   1.0   
173                             2017-07-18 10:51:30.195   2.0   
187                             2017-07-18 10:51:30.195   3.0   
201                             2017-07-18 10:51:30.195   4.0   
215                             2017-07-18 11:39:46.221   0.0   
229                             2017-07-18 11:39:46.221   1.0   
243                             2017-07-18 11:39:46.221   2.0   
257                             2017-07-18 11:39:46.221   3.0   
271                             2017-07-18 11:39:46.221   4.0   
285                             2017-07-18 14:25:46.769   0.0   
299                             2017-07-18 14:25:46.769   1.0   
313                             2017-07-18 14:25:46.769   2.0   
327                             2017-07-18 14:25:46.769   3.0   
341                             2017-07-18 14:25:46.769   4.0   
355                             2017-07-18 14:49:08.831   0.0   
369                             2017-07-18 14:49:08.831   1.0   
383                             2017-07-18 14:49:08.831   2.0   
397                             2017-07-18 14:49:08.831   3.0   
411                             2017-07-18 14:49:08.831   4.0   
...                                                 ...   ...   
4275                            2017-08-24 13:10:20.476   5.0   
4289                            2017-08-24 13:10:20.476   6.0   
4303                            2017-08-24 13:10:20.476   7.0   
4317                            2017-08-24 13:10:20.476   8.0   
4331                            2017-08-24 13:10:20.476   9.0   
4345                            2017-08-24 16:57:43.540   0.0   
4359                            2017-08-24 16:57:43.540   1.0   
4373                            2017-08-24 16:57:43.540   2.0   
4387                            2017-08-24 16:57:43.540   3.0   
4401                            2017-08-24 16:57:43.540   4.0   
4415                            2017-08-25 12:54:07.934   0.0   
4429                            2017-08-25 12:54:07.934   1.0   
4443                            2017-08-25 12:54:07.934   2.0   
4457                            2017-08-25 12:54:07.934   3.0   
4471                            2017-08-25 12:54:07.934   4.0   
4485                            2017-08-25 12:54:07.934   5.0   
4499                            2017-08-25 12:54:07.934   6.0   
4513                            2017-08-25 12:54:07.934   7.0   
4527                            2017-08-25 12:54:07.934   8.0   
4541                            2017-08-25 12:54:07.934   9.0   
4555                            2017-08-25 13:05:14.062   0.0   
4569                            2017-08-25 13:05:14.062   1.0   
4583                            2017-08-25 13:05:14.062   2.0   
4597                            2017-08-25 13:05:14.062   3.0   
4611                            2017-08-25 13:05:14.062   4.0   
4625                            2017-08-25 13:05:14.062   5.0   
4639                            2017-08-25 13:05:14.062   6.0   
4653                            201

## flatten column names

In [18]:
new_names2 = [tuple([c[0], c[2] or c[1], None]) for c in ne2.columns.values]
ne2.columns = pandas.MultiIndex.from_tuples(new_names2).droplevel(2)
ne2

set point index                                      \
                       date    id    e2    e3    e4    e5    e6    e7    e8   
0   2017-07-14 12:31:24.194   0.0     6     7     8     9    10    11    12   
1   2017-07-14 12:31:24.194   1.0    20    21    22    23    24    25    26   
2   2017-07-14 12:31:24.194   2.0    34    35    36    37    38    39    40   
3   2017-07-14 12:31:24.194   3.0    48    49    50    51    52    53    54   
4   2017-07-14 12:31:24.194   4.0    62    63    64    65    66    67    68   
5   2017-07-18 10:31:22.263   0.0    76    77    78    79    80    81    82   
6   2017-07-18 10:31:22.263   1.0    90    91    92    93    94    95    96   
7   2017-07-18 10:31:22.263   2.0   104   105   106   107   108   109   110   
8   2017-07-18 10:31:22.263   3.0   118   119   120   121   122   123   124   
9   2017-07-18 10:31:22.263   4.0   132   133   134   135   136   137   138   
10  2017-07-18 10:51:30.195   0.0   146   147   148   149   150   151   152   
11  2017-07-18 10:51:30.195   1.0   160   161   162   163   164   165   166   
12  2017-07-18 10:51:30.195   2.0   174   175   176   177   178   179   180   
13  2017-07-18 10:51:30.195   3.0   188   189   190   191   192   193   194   
14  2017-07-18 10:51:30.195   4.0   202   203   204   205   206   207   208   
15  2017-07-18 11:39:46.221   0.0   216   217   218   219   220   221   222   
16  2017-07-18 11:39:46.221   1.0   230   231   232   233   234   235   236   
17  2017-07-18 11:39:46.221   2.0   244   245   246   247   248   249   250   
18  2017-07-18 11:39:46.221   3.0   258   259   260   261   262   263   264   
19  2017-07-18 11:39:46.221   4.0   272   273   274   275   276   277   278   
20  2017-07-18 14:25:46.769   0.0   286   287   288   289   290   291   292   
21  2017-07-18 14:25:46.769   1.0   300   301   302   303   304   305   306   
22  2017-07-18 14:25:46.769   2.0   314   315   316   317   318   319   320   
23  2017-07-18 14:25:46.769   3.0   328   329   330   331   332   333   334   
24  2017-07-18 14:25:46.769   4.0   342   343   344   345   346   347   348   
25  2017-07-18 14:49:08.831   0.0   356   357   358   359   360   361   362   
26  2017-07-18 14:49:08.831   1.0   370   371   372   373   374   375   376   
27  2017-07-18 14:49:08.831   2.0   384   385   386   387   388   389   390   
28  2017-07-18 14:49:08.831   3.0   398   399   400   401   402   403   404   
29  2017-07-18 14:49:08.831   4.0   412   413   414   415   416   417   418   
..                      ...   ...   ...   ...   ...   ...   ...   ...   ...   
305 2017-08-24 13:10:20.476   5.0  4276  4277  4278  4279  4280  4281  4282   
306 2017-08-24 13:10:20.476   6.0  4290  4291  4292  4293  4294  4295  4296   
307 2017-08-24 13:10:20.476   7.0  4304  4305  4306  4307  4308  4309  4310   
308 2017-08-24 13:10:20.476   8.0  4318  4319  4320  4321  4322  4323  4324   
309 2017-08-24 13:10:20.476   9.0  4332  4333  4334  4335  4336  4337  4338   
310 2017-08-24 16:57:43.540   0.0  4346  4347  4348  4349  4350  4351  4352   
311 2017-08-24 16:57:43.540   1.0  4360  4361  4362  4363  4364  4365  4366   
312 2017-08-24 16:57:43.540   2.0  4374  4375  4376  4377  4378  4379  4380   
313 2017-08-24 16:57:43.540   3.0  4388  4389  4390  4391  4392  4393  4394   
314 2017-08-24 16:57:43.540   4.0  4402  4403  4404  4405  4406  4407  4408   
315 2017-08-25 12:54:07.934   0.0  4416  4417  4418  4419  4420  4421  4422   
316 2017-08-25 12:54:07.934   1.0  4430  4431  4432  4433  4434  4435  4436   
317 2017-08-25 12:54:07.934   2.0  4444  4445  4446  4447  4448  4449  4450   
318 2017-08-25 12:54:07.934   3.0  4458  4459  4460  4461  4462  4463  4464   
319 2017-08-25 12:54:07.934   4.0  4472  4473  4474  4475  4476  4477  4478   
320 2017-08-25 12:54:07.934   5.0  4486  4487  4488  4489  4490  4491  4492   
321 2017-08-25 12:54:07.934   6.0  4500  4501  4502  4503  4504  4505  4506   
322 2017-08-25 12:54:07.934   7.0  4514  4515  4516  4517  4518  4519  452

## merge parts

In [19]:
joined_df = base_df.join(ne2[['natural_enemy_count']]).join(aphid_df)
joined_df.columns = joined_df.columns.map('_'.join)
joined_df = joined_df.reindex(columns=[
    'set_date',
    'point_id',
    'observation_id',
    'observation_complete',
    'observation_disabled',
    'observation_aphid_subtotal',
    'point_natural_enemy_subtotal',
    'natural_enemy_count_e1',
    'natural_enemy_count_e2',
    'natural_enemy_count_e3',
    'natural_enemy_count_e4',
    'natural_enemy_count_e5',
    'natural_enemy_count_e6',
    'natural_enemy_count_e7',
    'natural_enemy_count_e8',
    'natural_enemy_count_e9',
    'aphid_count_a1',
    'aphid_count_a2',
    'aphid_count_a3',
    'set_obsName',
    'set_desc',
    'set_growthStage',
    'field_client_company',
    'field_client_displayText',
    'field_client_fname',
    'field_client_lname',
    'field_client_name',
    'field_crop',
    'field_date',
    'field_desc',
    'field_image',
    'field_name',
])
joined_df.sort_index()

,set_date,point_id,observation_id,observation_complete,observation_disabled,observation_aphid_subtotal,point_natural_enemy_subtotal,natural_enemy_count_e1,natural_enemy_count_e2,natural_enemy_count_e3,...,field_client_company,field_client_displayText,field_client_fname,field_client_lname,field_client_name,field_crop,field_date,field_desc,field_image,field_name
0,2017-07-14 12:31:24.194,0.0,0.0,NaN,NaN,0.0,NaN,1.0,1.0,0.0,...,AAFC SRDC,"AAFC SRDC, Tyler Wist",Tyler,Wist,Tyler Wist,CROPS.WHEAT,2017-07-14 12:30:31.587,Next to soybeans and canola,img/wheat.png,Sef wheat
1,2017-07-14 12:31:24.194,0.0,1.0,NaN,NaN,0.0,NaN,0.0,0.0,0.0,...,AAFC SRDC,"AAFC SRDC, Tyler Wist",Tyler,Wist,Tyler Wist,CROPS.WHEAT,2017-07-14 12:30:31.587,Next to soybeans and canola,img/wheat.png,Sef wheat
2,2017-07-14 12:31:24.194,0.0,2.0,NaN,NaN,0.0,NaN,0.0,0.0,0.0,...,AAFC SRDC,"AAFC SRDC, Tyler Wist",Tyler,Wist,Tyler Wist,CROPS.WHEAT,2017-07-14 12:30:31.587,Next to soybeans and canola,img/wheat.png,Sef wheat
3,2017-07-14 12:31:24.194,0.0,3.0,NaN,NaN,0.0,NaN,0.0,0.0,0.0,...,AAFC SRDC,"AAFC SRDC, Tyler Wist",Tyler,Wist,Tyler Wist,CROPS.WHEAT,2017-07-14 12:30:31.587,Next to soybeans and canola,img/wheat.png,Sef wheat
4,2017-07-14 12:31:24.194,0.0,4.0,NaN,NaN,0.0,NaN,0.0,0.0,0.0,...,AAFC SRDC,"AAFC SRDC, Tyler Wist",Tyler,Wist,Tyler Wist,CROPS.WHEAT,2017-07-14 12:30:31.587,Next to soybeans and canola,img/wheat.png,Sef wheat
5,2017-07-14 12:31:24.194,0.0,5.0,1.0,NaN,NaN,2.0,0.0,0.0,0.0,...,AAFC SRDC,"AAFC SRDC, Tyler Wist",Tyler,Wist,Tyler Wist,CROPS.WHEAT,2017-07-14 12:30:31.587,Next to soybeans and canola,img/wheat.png,Sef wheat
14,2017-07-14 12:31:24.194,1.0,0.0,NaN,NaN,0.0,NaN,0.0,0.0,0.0,...,AAFC SRDC,"AAFC SRDC, Tyler Wist",Tyler,Wist,Tyler Wist,CROPS.WHEAT,2017-07-14 12:30:31.587,Next to soybeans and canola,img/wheat.png,Sef wheat
15,2017-07-14 12:31:24.194,1.0,1.0,NaN,NaN,0.0,NaN,0.0,0.0,0.0,...,AAFC SRDC,"AAFC SRDC, Tyler Wist",Tyler,Wist,Tyler Wist,CROPS.WHEAT,2017-07-14 12:30:31.587,Next to soybeans and canola,img/wheat.png,Sef wheat
16,2017-07-14 12:31:24.194,1.0,2.0,NaN,NaN,0.0,NaN,0.0,0.0,0.0,...,AAFC SRDC,"AAFC SRDC, Tyler Wist",Tyler,Wist,Tyler Wist,CROPS.WHEAT,2017-07-14 12:30:31.587,Next to soybeans and canola,img/wheat.png,Sef wheat
17,2017-07-14 12:31:24.194,1.0,3.0,NaN,NaN,0.0,NaN,0.0,0.0,0.0,...,AAFC SRDC,"AAFC SRDC, Tyler Wist",Tyler,Wist,Tyler Wist,CROPS.WHEAT,2017-07-14 12:30:31.587,Next to soybeans and canola,img/wheat.png,Sef wheat


# save file

In [20]:
with pandas.ExcelWriter('data/real/2017/CAM data from iPads/out_df.xlsx') as file_writer:
    joined_df.to_excel(file_writer, sheet_name='2017 - CAM', freeze_panes=(1,0))
    file_writer.save()